# Исследование надёжности заёмщиков

## Описание проекта:

Заказчик — кредитный отдел банка. Нужно разобраться, **влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок**. Входные данные от банка — статистика о платёжеспособности клиентов.

## Исходные данные и их описание: 

* *children* — количество детей в семье;
* *days_employed* — общий трудовой стаж в днях
* *dob_years* — возраст клиента в годах
* *education* — уровень образования клиента
* *education_id* — идентификатор уровня образования
* *family_status* — семейное положение
* *family_status_id* — идентификатор семейного положения
* *gender* — пол клиента
* *income_type* — тип занятости
* *debt* — имел ли задолженность по возврату кредитов
* *total_income* — ежемесячный доход
* *purpose* — цель получения кредита

In [1]:
import pandas as pd                      # импорт библиотеки pandas
import numpy as np                       # импорт библиотеки pandas

data = pd.read_csv('/datasets/data.csv') # чтение файла с данными и сохранение в data

data.head(10)                            # получение первых 10 строк таблицы data

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [2]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


На основании пулученной информации можно сделать следующий предварительные выводы:
1. В полученном датафрейме есть пропуски и их нужно исправить;
2. В данных есть дубликаты;
3. Данные в колонке **days_employed** (общий трудовой стаж в днях)смотрятся странно и лучше использовать другой формат данных - целочисленный + значения есть как положительные, так и отрицательные 
4. Колонка **education_id** вызывает некоторые вопросы
5. Колонка **total_income** неудобно воспринимать
6. Колонка **purpose** нужно привести к одному виду. Есть фразы с одинаковой сутью, но записанные по разному - нужно навести порядок.

## Предобработка данных

In [3]:
# Посмотрим на названия колонок
data.columns.to_list()                     # перечень названий столбцов таблицы df

['children',
 'days_employed',
 'dob_years',
 'education',
 'education_id',
 'family_status',
 'family_status_id',
 'gender',
 'income_type',
 'debt',
 'total_income',
 'purpose']

### Поиск грубых ошибок

В названия колонок, отсутствую пробелы, названия записаны в хорошем стиле и все символы строчные.

**Но:**


**3.1.1.** Текстовых столбцах информация записана с использование разных регистров, приведем все к единому стилю.

In [4]:
def change_registr(data_series):
    '''
    Функция change_registr нужна для приведения всех строковых колонок к единому стилю
    data_series : Series
    ts          : Series
    '''
    ts = data_series.str.lower()
    return ts

# The list of text columns fro data
text_columns = ['education', 'family_status', 
                'gender'   , 'income_type'  ,
                'purpose'  ]              

# The cycle in text_columns
for column in text_columns:
    data[column] = change_registr(data[column])

# Display the new data
data.head(10) 

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,f,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,f,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,среднее,1,женат / замужем,0,m,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,m,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,f,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,m,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,f,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,среднее,1,женат / замужем,0,m,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,высшее,0,гражданский брак,1,f,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,m,сотрудник,0,144425.938277,покупка жилья для семьи


**3.1.2.** Необходимо проверить наличие ошибок или странных значений в данных и по возможности их исправить:

Получим полный список колонок и используем его для поиска уникальных значений и количества значений для каждой уникальной категории. При это колонки **days_employed**, **total_income** и **purpose** на данном моменте пока рассматривать не будем, поскольку в этих колонках возможно большое количество различных вариантов. 

In [5]:
data.columns # 

Index(['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose'],
      dtype='object')

In [6]:
column_list = ['children'     , 'dob_years'       , 'education', 'education_id',
               'family_status', 'family_status_id', 'gender'   , 'income_type' ,
               'debt'         ]

for col_name in column_list:
    print(f'Уникальные значения в столбце: {col_name}', data[col_name].unique(), '\n')
    display(data.groupby(col_name)[col_name].count().head(10))

Уникальные значения в столбце: children [ 1  0  3  2 -1  4 20  5] 



children
-1        47
 0     14149
 1      4818
 2      2055
 3       330
 4        41
 5         9
 20       76
Name: children, dtype: int64

Уникальные значения в столбце: dob_years [42 36 33 32 53 27 43 50 35 41 40 65 54 56 26 48 24 21 57 67 28 63 62 47
 34 68 25 31 30 20 49 37 45 61 64 44 52 46 23 38 39 51  0 59 29 60 55 58
 71 22 73 66 69 19 72 70 74 75] 



dob_years
0     101
19     14
20     51
21    111
22    183
23    254
24    264
25    357
26    408
27    493
Name: dob_years, dtype: int64

Уникальные значения в столбце: education ['высшее' 'среднее' 'неоконченное высшее' 'начальное' 'ученая степень'] 



...

gender
f      14236
m       7288
xna        1
Name: gender, dtype: int64

Уникальные значения в столбце: income_type ['сотрудник' 'пенсионер' 'компаньон' 'госслужащий' 'безработный'
 'предприниматель' 'студент' 'в декрете'] 



income_type
безработный            2
в декрете              1
госслужащий         1459
компаньон           5085
пенсионер           3856
предприниматель        2
сотрудник          11119
студент                1
Name: income_type, dtype: int64

Уникальные значения в столбце: debt [0 1] 



debt
0    19784
1     1741
Name: debt, dtype: int64

**3.1.3.** В результате проверки удалось выявить следующие явные проблемы в данных:

* В колонке **children** - присутствуют странные значения, например есть значения: **-1 ребенок** - 47 случаев и 20 случаев когда у человека **20 и более детей**. Предположу, что эти значения неправильные. 


* В колонке **dob_years** -  присутствуют странные значения, например есть значения **когда возвраст человека указан неверно**. По результатам проверки найден 101 случай когда возвраст равен 0. Что является ошибочной информацией; 


* В колонке **gender** -  присутствуют странные значения, например есть значениe: **xna** - что смотриться странным;


+ Колонки **education** и **education_id** не содержат грубых ошибок и хорошо согласуются друг с другом;


+ Колонки **family_status** и **family_status_id** не содержат грубых ошибок и хорошо согласуются друг с другом;


+ Колонки **income_type** и **debt** не содержат грубых ошибок.

Исправим ошибки в колонках **children**, **dob_years** и **gender**:

In [7]:
data['children']  = data['children'].replace(-1, 1)
data['children']  = data['children'].replace(20, 2)
data['dob_years'] = data['dob_years'].replace(0, np.nan)

# Определю индекс строки
#index_del_rows = data.loc[data['gender'] == 'xna']
#print (index_del_rows)

# Удаляю по индексу строки и делаю переиндексирование
data = data.drop(labels=[10701], axis=0).reset_index(drop=True) 

Делаю дополнительную проверку и смотрю удалось ли исправить ошибки

In [8]:
column_list = ['children', 'dob_years', 'gender']

for col_name in column_list:
    print(f'Уникальные значения в столбце: {col_name}', data[col_name].unique(), '\n')
    display(data.groupby(col_name)[col_name].count().head(10))

Уникальные значения в столбце: children [1 0 3 2 4 5] 



children
0    14148
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

Уникальные значения в столбце: dob_years [42. 36. 33. 32. 53. 27. 43. 50. 35. 41. 40. 65. 54. 56. 26. 48. 24. 21.
 57. 67. 28. 63. 62. 47. 34. 68. 25. 31. 30. 20. 49. 37. 45. 61. 64. 44.
 52. 46. 23. 38. 39. 51. nan 59. 29. 60. 55. 58. 71. 22. 73. 66. 69. 19.
 72. 70. 74. 75.] 



dob_years
19.0     14
20.0     51
21.0    111
22.0    183
23.0    254
24.0    263
25.0    357
26.0    408
27.0    493
28.0    503
Name: dob_years, dtype: int64

Уникальные значения в столбце: gender ['f' 'm'] 



gender
f    14236
m     7288
Name: gender, dtype: int64

Да, часть ошибок удалось исправить. Но теперь в столбце **dob_years** появился nan. Настало время занятся пропусками.

### Пропуски значений

Считаем пропуски и определяем сколько всего пропущенных значений в наших столбцах.

In [9]:
print(data.isnull().sum()) # подсчёт пропусков

children               0
days_employed       2174
dob_years            101
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64


Пропуски есть в трех столбцах **days_employed**, **total_income** и **dob_years**. К основным причинам возникновения пропусков относятся:
* Пропуски в колонке **dob_years** были созданы мною, для того чтобы исправить некорректные значения возвраста. Пропуски легко исправить на основе данных из колонки **income_type**.

In [10]:
print('Пропуски до:', data['dob_years'].isna().sum(), '\n')

for group_age in data['income_type'].unique():
    median_value = data.loc[data['income_type'] == group_age, 'dob_years' ].median()
    #print(group_age, median_value)
    data.loc[(data['dob_years'].isna()) & (data['income_type'] == group_age), 'dob_years'] = median_value
    
print('Пропуски после:', data['dob_years'].isna().sum(), '\n')

Пропуски до: 101 

Пропуски после: 0 



* Количество пропусков в столбцах **days_employed** и **total_income** одинаковое, что позволяет предположить наличие связи между этими показателями. Например: нет работы --> нет зарплаты, либо клиент отказался предоставить эти сведения. 

Исходя из названия колонок и логики можно сделать следующее предположение, что заполнять пропуски в колонке **total_income** нужно в зависимости от значений в **days_employed**. Поскольку зарплата зависит от стажа и образования. 

В свою очередь, заполнить пропуски в колонке **days_employed** можно в зависимости от **dob_years**, **education**, **gender**. Дополнительно, можноо привлечь информацию из колонки **family_status**, но я этого делать не буду. Поскольку с увеличением количества параметров на основе, которых будет делаться коррекция увеличивается шанс, что количество пропусков без исправлений увеличится.


In [11]:
# Часть значений имеют отрицательные значения, часть положительные. В целом нужен единый стиль.
# Поскольку стаж не может быть отрицательным приведем все к положительным значениям.
data['days_employed'] = abs(data['days_employed'])


print('Пропуски до:', data['days_employed'].isna().sum(), '\n')

medians = (data.groupby(['dob_years',
                         'education', 
                         'gender'   ])).agg({'days_employed':'median'}).rename(columns = {'days_employed':'median_days_employed'})

data = data.merge(medians, on = ['dob_years',
                                 'education',
                                 'gender'   ])

#data[['dob_years'  , 'education',  'gender',
#      'income_type', 'days_employed', 'median_days_employed']][data['days_employed'].isna()].head()

data.loc[data['days_employed'].isna(), 'days_employed'] = data.loc[data['days_employed'].isna(), 'median_days_employed']

print('Пропуски после:', data['days_employed'].isna().sum(), '\n')


Пропуски до: 2174 

Пропуски после: 3 



После заполнения пропусков остались 3 строчки, где исправить пропуски не получилось. Удалим данные строчки в зависимости от их индекса. (Примечание: данная часть является не оптимальной, по возможности исправить).

In [12]:
print('before', data.shape)
# Определю индекс строк
index_del_rows = data.loc[data['days_employed'].isna()]
#print(index_del_rows.index)


# Удаляю по индексу строки и делаю переиндексирование
for index in range(len(index_del_rows)):
    data = data.drop(labels = [21479, 21510, 21516], axis=0).reset_index(drop=True) 
    
print('after', data.shape)

before (21524, 13)
after (21515, 13)


Дополнительные действия для приведения колонки **days_employed** в удовлетворительный вид:

In [13]:
# Данные в столбце  представлены в формате **float64**, который не удобно использовать 
# для подсчета количества дней. Изменим тип на целочисленный: 
data['days_employed'] = data['days_employed'].astype('int')

# Отсортируем значения в столбце days_employed по убыванию и посмотрим все ли хорошо:
display(data.sort_values('days_employed', ascending=False).head(10))

# Как оказалось есть значения, которые сильно отличаются от остальных

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,median_days_employed
13238,0,401755,56.0,среднее,1,вдовец / вдова,2,f,пенсионер,0,176278.441171,ремонт жилью,335974.254438
20731,0,401715,69.0,высшее,0,не женат / не замужем,4,f,пенсионер,0,57390.256908,получение образования,370420.179001
6034,1,401675,61.0,среднее,1,женат / замужем,0,f,пенсионер,0,126214.519212,операции с жильем,347226.626654
19064,0,401674,60.0,среднее,1,женат / замужем,0,m,пенсионер,0,325395.724541,автомобили,5635.422839
6038,0,401663,61.0,среднее,1,гражданский брак,1,f,пенсионер,0,48286.441362,свадьба,347226.626654
13210,0,401635,56.0,среднее,1,женат / замужем,0,f,пенсионер,0,48242.322502,покупка недвижимости,335974.254438
3550,0,401619,63.0,среднее,1,гражданский брак,1,f,пенсионер,0,51449.788325,сыграть свадьбу,362963.961475
9961,0,401614,59.0,среднее,1,женат / замужем,0,f,пенсионер,0,152769.694536,покупка жилья для сдачи,353311.040246
13280,0,401591,56.0,среднее,1,в разводе,3,f,пенсионер,0,39513.517543,получение дополнительного образования,335974.254438
13671,0,401590,58.0,среднее,1,женат / замужем,0,f,пенсионер,0,175306.312902,образование,340243.384325


Исправим странные значения в колонке **days_employed**, которые связаны с другим типом хранения временных данных (скорее всего используются данные из другой базы данных). Новые данные добавим в колонку **new_days_employed**.  

Следует отметить, что некоторые значения вызывают вопросы и после коррекции (значения больше 15000 дней). Но исправить или уточнить текущие значения не представляется возможным.

In [14]:
def change_year(row):
    MIN_STAG = 16                   # Минимальный возвраст с которого можно начать работать,
                                    # получив при этом среднее образование
        
    NUM_DAYS = 365                  # Количество дней в году, без учета високосных лет
    
    year = row['dob_years']
    stag = row['days_employed']
    
    if stag > ((year - MIN_STAG) * NUM_DAYS):
        return(stag / 24)
    else:
        return stag
    
data['new_days_employed'] = data.apply(change_year, axis = 1)               # Применим функцию для коррекции
data['new_days_employed'] = data['new_days_employed'].astype('int')         # Изменим тип данных на целочисленный

In [15]:
display(data.sort_values('days_employed', ascending=False).head(10))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,median_days_employed,new_days_employed
13238,0,401755,56.0,среднее,1,вдовец / вдова,2,f,пенсионер,0,176278.441171,ремонт жилью,335974.254438,16739
20731,0,401715,69.0,высшее,0,не женат / не замужем,4,f,пенсионер,0,57390.256908,получение образования,370420.179001,16738
6034,1,401675,61.0,среднее,1,женат / замужем,0,f,пенсионер,0,126214.519212,операции с жильем,347226.626654,16736
19064,0,401674,60.0,среднее,1,женат / замужем,0,m,пенсионер,0,325395.724541,автомобили,5635.422839,16736
6038,0,401663,61.0,среднее,1,гражданский брак,1,f,пенсионер,0,48286.441362,свадьба,347226.626654,16735
13210,0,401635,56.0,среднее,1,женат / замужем,0,f,пенсионер,0,48242.322502,покупка недвижимости,335974.254438,16734
3550,0,401619,63.0,среднее,1,гражданский брак,1,f,пенсионер,0,51449.788325,сыграть свадьбу,362963.961475,16734
9961,0,401614,59.0,среднее,1,женат / замужем,0,f,пенсионер,0,152769.694536,покупка жилья для сдачи,353311.040246,16733
13280,0,401591,56.0,среднее,1,в разводе,3,f,пенсионер,0,39513.517543,получение дополнительного образования,335974.254438,16732
13671,0,401590,58.0,среднее,1,женат / замужем,0,f,пенсионер,0,175306.312902,образование,340243.384325,16732


Исправим пропуски в колонке **total_income** в зависимости от **new_days_employed** и **education**. 

К сожалению из большого количества различных вариантов в **new_days_employed** заполнить сразу все пропуски в **total_income** используя только ранее используемый метод не получается. Остается большое количество пропусков. Поэтому, чтобы не потерять данные дополнительно создадим столбец **days_employed_kateg**. В стобце содержится текстовая информация о трудовом стаже разбитом на категории упростим задачу для Python.

In [16]:
def stag_by_category(row):
    ONE_YEAR = 365
    stag = row['new_days_employed']
    
    if 0 < stag < (ONE_YEAR * 5):
        return 'Стаж 1 - 5 лет'
    elif (ONE_YEAR * 5) < stag < (ONE_YEAR * 10):
        return 'Стаж 5 - 10 лет'
    elif (ONE_YEAR * 10) < stag < (ONE_YEAR * 15):
        return 'Стаж 10 - 15 лет'
    elif (ONE_YEAR * 15) < stag < (ONE_YEAR * 20):
        return 'Стаж 15 - 20 лет'
    elif (ONE_YEAR * 20) < stag < (ONE_YEAR * 25):
        return 'Стаж 20 - 25 лет'
    elif (ONE_YEAR * 25) < stag < (ONE_YEAR * 30):
        return 'Стаж 25 - 30 лет'
    else:
        return 'более 30 лет'
    
data['days_employed_kateg'] = data.apply(stag_by_category, axis = 1)

Наконец можно убрать все пропуски из столбца **total_income**

In [17]:
medians = (data.groupby(['days_employed_kateg',
                        'education'])).agg({'total_income':'median'}).rename(columns = {'total_income':'median_total_income'})

data = data.merge(medians, on = ['days_employed_kateg',
                                 'education'])

data.loc[data['total_income'].isna(), 'total_income'] = data.loc[data['total_income'].isna(), 'median_total_income']

data['total_income'] = data['total_income'].astype('int')

In [18]:
# Check nan values
print(data.isnull().sum())

children                0
days_employed           0
dob_years               0
education               0
education_id            0
family_status           0
family_status_id        0
gender                  0
income_type             0
debt                    0
total_income            0
purpose                 0
median_days_employed    0
new_days_employed       0
days_employed_kateg     0
median_total_income     0
dtype: int64


Отлично, больше нет пропусков, можно приступить к следующему этапу.

### Дубликаты

*Задача найти дубликаты и удалить их*. **Основных причин** появления дубликатов может быть несколько:
+ При объединении разных быз данных, один и тотже человек мог быть добавлен несколько раз.
+  В какой-то момент при заполнения человеком соответствующих данных компьютер завис после нажатия кнопки отправить, после чего выдал ошибку. При этом данные были отправлены в базу данных, но человек этого незнал и внес свои данные повторно.

**3.3.1** Считаем явные дубликаты в таблице

In [19]:
print(data.duplicated().sum())# подсчёт явных дубликатов

71


**3.3.2** Вызываем специальный метод pandas и удаляем явные дубликаты

In [20]:
data = data.drop_duplicates().reset_index(drop = True)  # удаление явных дубликатов
                                                        # (с удалением старых индексов и формированием новых)

**3.3.3** Убеждаемся, что все явные дубликаты в таблице удалены

In [21]:
print(data.duplicated().sum()) # проверка на отсутствие дубликатов

0


**3.3.4** Поиск неявных дубликатов, или некорректных названий был выполнен ранее, поэтому в данном разделе данный поиск повторно выполнятся не будет.

### Лемматизация

In [22]:
from pymystem3 import Mystem        # 
from collections import Counter

m = Mystem() 

lemma = m.lemmatize(' '.join(data['purpose']))
print(Counter(lemma)) 

Counter({' ': 54999, 'недвижимость': 6345, 'покупка': 5894, 'жилье': 4459, 'автомобиль': 4304, 'образование': 4013, 'с': 2917, 'операция': 2602, 'свадьба': 2323, 'свой': 2228, 'на': 2222, 'строительство': 1877, 'высокий': 1374, 'получение': 1314, 'коммерческий': 1310, 'для': 1289, 'жилой': 1230, 'сделка': 941, 'дополнительный': 906, 'заниматься': 904, 'подержать': 837, 'проведение': 768, 'сыграть': 764, 'сдача': 651, 'семья': 638, 'собственный': 634, 'со': 626, 'ремонт': 607, 'приобретение': 460, 'профильный': 436, 'подержанный': 127, '\n': 1})


На основе лемматизации определили, что есть несколько наиболее частотных слов, которые мы можем использовать для создания специальных категорий (Причем лучше использовать существительные, а не глаголы):
1. недвижимость и жилье;
2. автомобиль;
3. образование;
4. операция;
5. свадьба

In [23]:
def purpose_by_category(row):
    purpose = row['purpose']
    
    if ('жиль' in purpose) or ('недвиж' in purpose):
        return 'недвижимость'
    elif ('авто' in purpose):
        return 'авто'
    elif ('образов' in purpose):
        return 'образование'
    #elif ('опер' in purpose):
    #    return 'операция'
    else:
        return 'свадьба'

    
data['new_purpose'] = data.apply(purpose_by_category, axis = 1)

Остался последний столбец, который нужно обратотать, прежде чем приступить к ответам на вопросы. А именно столбец с зарплатами. Создадим категории:

In [26]:
# задам уровни дохода 

def salary_by_category(row):
    salary = row['total_income']
    
    if 0 < salary <= 25000:
        return 'очень бедные'
    elif 25000 < salary <= 50000:
        return 'бедные'
    elif 50000 < salary <= 100000:
        return 'средний класс'
    elif 100000 < salary <= 200000:
        return 'почти богаты'
    else:
        return 'богатые'

    
data['new_salary'] = data.apply(salary_by_category, axis = 1)

В результате предобработки данных были созданы три  дополнительных столбца:**days_employed_kateg**, **new_purpose** и **new_salary**,приступим к ответам на вопросы.

## Ответьте на вопросы

### Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [31]:
report = data.groupby('children').agg({'debt':['mean', 'count']})

report = report.rename(columns = {'debt' :'Описательная статистика'}, level = 0)
report = report.rename(columns = {'count':'Количество заемщиков',
                                  'mean' :'%невозврата'        }, level = 1)

report.index = report.index.rename('Количество детей')

report.style.format(formatter={('Описательная статистика', '%невозврата'         ): "{:.2%}",
                                        ('Описательная статистика', 'Количество заемщиков'): "{:.1f}"})



**Вывод**: Определить какую либо строгую связь между наличием детей и возвратом кредита в срок определить довольно сложно. Процентное соотношение приблизительно равно. При этом семьи с 5 детьми каждый раз возвращали кредиты вовремя. 

### Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [34]:
report = data.groupby('family_status').agg({'debt':['mean', 'count']})

report = report.rename(columns = {'debt' :'Описательная статистика'}, level = 0)
report = report.rename(columns = {'count':'Количество заемщиков',
                                  'mean' :'%невозврата'        }, level = 1)

report.index = report.index.rename('Семейное положение')

report.style.format(formatter={('Описательная статистика', '%невозврата'         ): "{:.2%}",
                               ('Описательная статистика', 'Количество заемщиков'): "{:.1f}"})


**Вывод**: Выделить строгую связь по прежнему проблематично. Данные показывают, что есть две категории граждан (гражданский брак и не женат / не замужен), где процент невозврата выше, но при этом отличия в 3 % непозволяет с уверенностью дать ответ на вопрос, что семейные пары возвращают кредит всегда в срок.

### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [35]:
# Group data
report = data.groupby('new_salary').agg({'debt':['mean', 'count']})

# Rename columns 
report = report.rename(columns = {'debt' :'Описательная статистика'}, level = 0)
report = report.rename(columns = {'count':'Количество заемщиков',
                                  'mean' :'%невозврата'        }, level = 1)

# Rename index
report.index = report.index.rename('Уровень дохода')

# Change output format --> only view
report.style.format(formatter={('Описательная статистика', '%невозврата'         ): "{:.2%}",
                               ('Описательная статистика', 'Количество заемщиков'): "{:.1f}"})

**Вывод**: Данные показывают, что есть связь между % невозрата и уровнем зарплаты. Так в 8 случае с крайне низким доходом, наблюдается 12.5% невозвратов. При этом люди с низким доходом старатся выплачивать кредиты вовремя. 

### Как разные цели кредита влияют на его возврат в срок?

In [37]:
report = data.groupby('new_purpose').agg({'debt':['mean', 'count']})

report = report.rename(columns = {'debt' :'Описательная статистика'}, level = 0)
report = report.rename(columns = {'count':'Количество заемщиков',
                                  'mean' :'%невозврата'        }, level = 1)

report.index = report.index.rename('Цель кредита')

report.style.format(formatter={('Описательная статистика', '%невозврата'         ): "{:.2%}",
                               ('Описательная статистика', 'Количество заемщиков'): "{:.1f}"})



**Вывод**: Наибольший процент невозврата кредита в срок наблюдается при взятии кредита для образовательных целей. Данный факт можно объяснить тем, что получение образования стоит денег, а получения выгод из наличия образования может проявлятся не сразу. Наибольшее количество обращений связано с недвижимостью, при этом люди стараются выплачивать такие кредиты в срок и процент невозврата у данной категории наименьший. Данный факт также можно объяснить тем, что на покупку недвижимости люди решаются имея стабильную работу и зарплату (в большинстве случаев), что и позволяет им вовремя производить кредитный выплаты.

## Общий вывод

В заключении можно отметить, что все выше полученные выводы являются спорными и для однозначного ответа на поставленные вопросы требуется проведение дополнительного статистического анализа данных. Только группировки, явно недостаточно. При этом можно отметить, что наиболее надежными являются люди, которые берут кредит с целью покупки недвижимости. Связано это прежде всего с тем, что чаще всего необходимость покупки собственного жилья возникает у женатых людей с наличием детей. Кроме того, как правило, такие люди имеют стабильную работу и определенный уровень образования, которые может гарантировать им некоторую уверенность. Поэтому кредиты на покупку недвижимости обладают наименьшим процентом невозврата.

Наибольший риск невозврата по кредитам может быть отмечен у не женатых людей, без детей, которые берут кредит с целью получения образования или покупки авто.Данный факт, также можно легко объяснить человеческим фактором. Как правило - это молодые люди, которые неимеют стабильного дохода, что и не позволяет им делать выплаты в срок. Такие кредиты являются более рискованными. 